In [21]:
import pandas as pd
import numpy as np
df1 = pd.read_csv("/content/submission.csv")

In [22]:
df1.head(2)

,#,title,submitted,last updated,form fields,keywords,decision,notified,reviews sent,abstract
0,199,Li-Ion Battery State-of-Charge Estimation Base...,2022-12-15 09:35,1970-01-01 00:00,NaN,State of charge (SoC)\nLithium-ion battery (LI...,reject,yes,yes,The appropriate evaluation of the State of Cha...
1,428,Accessing Health Information from Social Media...,2022-11-27 14:44,1970-01-01 00:00,NaN,Social media\nHealth information\nSeeking\nJordan,reject,yes,yes,Background: Little research has been concentra...


In [23]:
abstracts = df1['abstract']

In [24]:
abstracts

0     The appropriate evaluation of the State of Cha...
1     Background: Little research has been concentra...
2     Wireless sensing is now the spine of diverse I...
3     The paper discusses the possibilities for deve...
4     This research addresses the problem of spaces ...
5     This paper presents the utilization of polymer...
6     Recently, small satellites such as Cube Satell...
7     The current research aimed to identify the eff...
8     Covid-19 has had a destructive influence on gl...
9     When employing pure hydrogen, its leakage pose...
10    We continuously visit pharmacies to buy medici...
11    Institutions of learning are beginning to embr...
12    This research explores machine learning use-pa...
13    The use of machine learning to forecast how na...
14    Some individuals in society suffer from diseas...
15    Introduction:  Diabetes therapeutic education ...
16    The rapidly increasing demand for mobile commu...
17    The integration of technology in education

In [32]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('stopwords')
nltk.download('punkt')

# Tokenizing the text
stopWords = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [33]:
summaries = []  # List to store summaries for each abstract

for abstract in abstracts:
    words = word_tokenize(abstract)

    # Creating a frequency table to keep the score of each word
    freqTable = dict()
    for word in words:
        word = word.lower()
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1

    # Creating a dictionary to keep the score of each sentence
    sentences = sent_tokenize(abstract)
    sentenceValue = dict()

    for sentence in sentences:
        for word, freq in freqTable.items():
            if word in sentence.lower():
                if sentence in sentenceValue:
                    sentenceValue[sentence] += freq
                else:
                    sentenceValue[sentence] = freq

    sumValues = 0
    for sentence in sentenceValue:
        sumValues += sentenceValue[sentence]

    # Average value of a sentence from the original text
    average = int(sumValues / len(sentenceValue))

    # Storing sentences into our summary.
    summary = ''
    for sentence in sentences:
        if sentence in sentenceValue and sentenceValue[sentence] > (1.2 * average):
            summary += " " + sentence

    summaries.append(summary)

# Print summaries for each abstract
for i, summary in enumerate(summaries, start=1):
    print(f"Summary for Abstract {i}:\n{summary}\n")


Summary for Abstract 1:
 The appropriate evaluation of the State of Charge (SoC) of the battery is critical for ensuring safety and avoiding potential malfunctions in battery-powered electric cars, cell phones, computers, and medical devices.

Summary for Abstract 2:
 The aim of this study was therefore to explore their use of SMAs (i.e., WhatsApp, Twitter, YouTube, Snapchat, Instagram, and Facebook) for accessing health information.

Summary for Abstract 3:
 In the Metaverse, the Internet of Things (IoT) can offer wireless and seamlessly integrated immersive digital experiences. The sensor node's computations and energy consumption are increased due to the noise in the signal, resulting in the sensor node's shorter lifespan. Theoretically, it is observed that the CRA minimizes delay to 86.26% compared to carry propagation adder (CPA) approaches. Additionally, compared to a typical R8BR multiplier, the experimental findings indicated delay, area and power reductions of 48.98%, 56.66%, 

In [34]:
df2 = pd.DataFrame({
    '#': df1['#'],
    'title': df1['title'],
    'abstract': df1['abstract'],
    'summary': summaries
})

In [35]:
df2['len_abstract'] = df2['abstract'].apply(len)
df2['len_summary'] = df2['summary'].apply(len)

In [36]:
df2.head()

,#,title,abstract,summary,len_abstract,len_summary
0,199,Li-Ion Battery State-of-Charge Estimation Base...,The appropriate evaluation of the State of Cha...,The appropriate evaluation of the State of Ch...,1109,218
1,428,Accessing Health Information from Social Media...,Background: Little research has been concentra...,The aim of this study was therefore to explor...,1068,169
2,563,Optimizing energy efficiencies of IoT-based wi...,Wireless sensing is now the spine of diverse I...,"In the Metaverse, the Internet of Things (IoT...",2015,561
3,700,Embracing the Metaverse: The Future of Islami...,The paper discusses the possibilities for deve...,"The study contemplates the practical, educati...",839,297
4,876,Space Voxelization with Generative Adversial N...,This research addresses the problem of spaces ...,This research addresses the problem of spaces...,1266,547


In [51]:
from nltk.stem.wordnet import WordNetLemmatizer
import string
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [52]:
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stopWords])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

summaries = df2['summary']
doc_clean = [clean(doc).split() for doc in summaries]

In [53]:
# Importing Gensim
import gensim
from gensim import corpora

In [54]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index.

dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [55]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

In [60]:
print(ldamodel.print_topics(num_topics=1, num_words=3))

[(0, '0.009*"food" + 0.007*"result" + 0.007*"research"')]


In [61]:
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
  print(topic)

(0, '0.009*"food" + 0.007*"result" + 0.007*"research" + 0.007*"method" + 0.006*"student"')
(1, '0.016*"field" + 0.013*"model" + 0.011*"degree" + 0.010*"mpa" + 0.008*"group"')
(2, '0.009*"metaverse" + 0.007*"security" + 0.007*"research" + 0.007*"data" + 0.006*"system"')


In [62]:
# Get a summary of all topics in each document
for i, doc_topics in enumerate(ldamodel[doc_term_matrix]):
    print(f"\nSummary {i+1} Topics:")
    for topic_num, relevance in doc_topics:
        print(f"Topic {topic_num}: {relevance * 100}% relevant")


Summary 1 Topics:
Topic 0: 96.66585326194763% relevant
Topic 1: 1.6409015282988548% relevant
Topic 2: 1.6932453960180283% relevant

Summary 2 Topics:
Topic 0: 95.83983421325684% relevant
Topic 1: 2.0942322909832% relevant
Topic 2: 2.065930888056755% relevant

Summary 3 Topics:
Topic 1: 98.72046709060669% relevant

Summary 4 Topics:
Topic 0: 1.25839589163661% relevant
Topic 1: 42.15661883354187% relevant
Topic 2: 56.58498406410217% relevant

Summary 5 Topics:
Topic 2: 98.73948693275452% relevant

Summary 6 Topics:
Topic 1: 98.62175583839417% relevant

Summary 7 Topics:
Topic 2: 98.12838435173035% relevant

Summary 8 Topics:
Topic 2: 98.08645248413086% relevant

Summary 9 Topics:
Topic 2: 98.7432599067688% relevant

Summary 10 Topics:
Topic 2: 98.0351209640503% relevant

Summary 11 Topics:
Topic 1: 98.11951518058777% relevant

Summary 12 Topics:
Topic 0: 97.91104197502136% relevant
Topic 1: 1.0292138904333115% relevant
Topic 2: 1.0597403161227703% relevant

Summary 13 Topics:
Topic 0: 3

In [95]:
import pandas as pd
import numpy as np
df3 = pd.read_csv("/content/review.csv")

In [96]:
df3.head()

,#,submission #,number,version,text,scores,total score,reviewer person #,date,time,attachment?
0,1,199,1,1,(OVERALL EVALUATION) Thank you for sending the...,Overall evaluation: -3\nReviewer's confidence: 3,-3,120.0,1/1/2023,20:10,no
1,2,199,2,1,(OVERALL EVALUATION) The paper presents work o...,Overall evaluation: -1\nReviewer's confidence: 4,-1,NaN,1/8/2023,9:35,no
2,3,428,1,1,(OVERALL EVALUATION) Although the paper target...,Overall evaluation: 0\nReviewer's confidence: 4,0,106.0,12/28/2022,7:58,no
3,4,428,2,1,(OVERALL EVALUATION) The paper examines the he...,Overall evaluation: -2\nReviewer's confidence: 5,-2,121.0,1/2/2023,15:38,no
4,5,563,1,1,(OVERALL EVALUATION) The paper is well written...,Overall evaluation: 2\nReviewer's confidence: 4,2,NaN,1/6/2023,16:25,no


In [97]:
import re

# Functions to remove noise
# Remove brackets
def remove_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

# Lowercase the text
def lower_cases(text):
    return text.lower()

# Remove special characters
def remove_char(text):
    pattern = r'[^a-zA-Z0-9\s]'
    text = re.sub(pattern, '', text)
    return text

# Remove noise (combine above functions)
def remove_noise(text):
    # Remove "(OVERALL EVALUATION)"
    text = re.sub(r'\(OVERALL EVALUATION\)', '', text)
    text = remove_brackets(text)
    text = lower_cases(text)
    text = remove_char(text)

    return text

# Call the function on predictors
df3['text'] = df3['text'].apply(remove_noise)


In [98]:
df3.head()

,#,submission #,number,version,text,scores,total score,reviewer person #,date,time,attachment?
0,1,199,1,1,thank you for sending the manuscript for cons...,Overall evaluation: -3\nReviewer's confidence: 3,-3,120.0,1/1/2023,20:10,no
1,2,199,2,1,the paper presents work on liion battery sta...,Overall evaluation: -1\nReviewer's confidence: 4,-1,NaN,1/8/2023,9:35,no
2,3,428,1,1,although the paper targets a very important s...,Overall evaluation: 0\nReviewer's confidence: 4,0,106.0,12/28/2022,7:58,no
3,4,428,2,1,the paper examines the health information sou...,Overall evaluation: -2\nReviewer's confidence: 5,-2,121.0,1/2/2023,15:38,no
4,5,563,1,1,the paper is well written and organized so i ...,Overall evaluation: 2\nReviewer's confidence: 4,2,NaN,1/6/2023,16:25,no


In [99]:
from nltk.stem.porter import PorterStemmer
def stem_words(text):
 ps = PorterStemmer()
 stem_list = [ps.stem(word) for word in text.split()]
 text = ''.join(ps.stem(word) for word in text)

 return text
df3['text'] = df3['text'].apply(stem_words)

In [100]:
# removing the stopwords from review
def remove_stopwords(text):
    # list to add filtered words from review
    filtered_text = []
    # verify & append words from the text to filtered_text list
    for word in text.split():
      if word not in stopWords:
        filtered_text.append(word)
    # add content from filtered_text list to new variable
    clean_review = filtered_text[:]
    # emptying the filtered_text list for new review
    filtered_text.clear()
    return clean_review

df3['text']=df3['text'].apply(remove_stopwords)

# join back all words as single paragraph
def join_back(text):
    return ' '.join(text)
df3['text'] = df3['text'].apply(join_back)

In [101]:
df3.head()

,#,submission #,number,version,text,scores,total score,reviewer person #,date,time,attachment?
0,1,199,1,1,thank sending manuscript consideration lt2023 ...,Overall evaluation: -3\nReviewer's confidence: 3,-3,120.0,1/1/2023,20:10,no
1,2,199,2,1,paper presents work liion battery stateofcharg...,Overall evaluation: -1\nReviewer's confidence: 4,-1,NaN,1/8/2023,9:35,no
2,3,428,1,1,although paper targets important subject handl...,Overall evaluation: 0\nReviewer's confidence: 4,0,106.0,12/28/2022,7:58,no
3,4,428,2,1,paper examines health information sought smas ...,Overall evaluation: -2\nReviewer's confidence: 5,-2,121.0,1/2/2023,15:38,no
4,5,563,1,1,paper well written organized accept without co...,Overall evaluation: 2\nReviewer's confidence: 4,2,NaN,1/6/2023,16:25,no


In [154]:
X = df3['text']
#transforming y to have only -1 for negative reveiws, 0 for neutral, and 1 for positive
y = df3['total score'].apply(lambda x: -1 if x < 0 else (1 if x > 0 else 0))

In [155]:
y

0    -1
1    -1
2     0
3    -1
4     1
     ..
93   -1
94    1
95    0
96    1
97    1
Name: total score, Length: 98, dtype: int64

In [156]:
# train set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [157]:
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer(ngram_range=(1,2))

x_vec=cv.fit_transform(X_train).toarray()

In [158]:
xt_vec=cv.transform(X_test).toarray()

In [159]:
from sklearn.naive_bayes import MultinomialNB
#this version of naive bayes is used for text classification

mn=MultinomialNB()

mn.fit(x_vec,y_train)

y_pred=mn.predict(xt_vec)

y_pred

array([ 1,  1,  1,  1,  1, -1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1])

In [160]:
y_test

62    0
40   -1
94    1
18    1
81    1
83   -1
64    1
42   -1
10    1
0    -1
31    1
75   -1
47    1
26   -1
44    1
4     1
22    0
12    1
90    1
73   -1
Name: total score, dtype: int64

In [161]:
from sklearn.metrics import accuracy_score
print("Accuracy", accuracy_score(y_test, y_pred))


Accuracy 0.65
